In [ ]:
import sys, time, os, asyncio, glob
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
import pickle as pkl
import pandas as pd
from astropy.time import Time, TimeDelta

from lsst_efd_client import EfdClient
from lsst.daf.butler import Butler
%matplotlib inline

In [ ]:
client = EfdClient('summit_efd')
butler = Butler('/repo/LATISS', collections="LATISS/raw/all")

In [ ]:
# Get the timestamps of the desired image:
expId = 2022112300044
mData = butler.get('raw.metadata', detector=0, exposure=expId)
start = Time(mData['DATE-BEG'])
end = Time(mData['DATE-END'])
print(start, end)

In [ ]:
for expId in range(2022112300016, 2022112300046):
    i = expId - 2022112300016
    print(expId, int((i - i%5) / 5))

In [ ]:
ux_means = []
ux_stds = []
uy_means = []
uy_stds = []
uz_means = []
uz_stds = []
fans = []

fan_speeds = [0.0,10.0,20.0,30.0,40.0,50.0]
for expId in range(2022112300016, 2022112300046):
    i = expId - 2022112300016
    fan_index = int((i - i%5) / 5)
    fans.append(fan_speeds[fan_index])
    mData = butler.get('raw.metadata', detector=0, exposure=expId)
    start = Time(mData['DATE-BEG'])
    end = Time(mData['DATE-END'])
    anemom_start = start - TimeDelta(30.0, format='sec')
    anemom_end = start + TimeDelta(30.0, format='sec')

    ane = await client.select_time_series('lsst.sal.ESS.airTurbulence', \
                                            ['*'],  anemom_start, anemom_end)
    ux_mean = ane['speed0'].values[0]
    ux_std = ane['speedStdDev0'].values[0]
    uy_mean = ane['speed1'].values[0]
    uy_std = ane['speedStdDev1'].values[0]
    uz_mean = ane['speed2'].values[0]
    uz_std = ane['speedStdDev2'].values[0]
    ux_means.append(ux_mean)
    ux_stds.append(ux_std)
    uy_means.append(uy_mean)
    uy_stds.append(uy_std)
    uz_means.append(uz_mean)
    uz_stds.append(uz_std)




In [ ]:

plt.figure(figsize=(8,8))
plt.subplots_adjust(hspace=0.5, wspace=0.5)
plt.subplot(3,2,1)
plt.title("Ux Mean")
plt.scatter(fans, ux_means)
plt.ylim(-0.5, 0.5)
plt.ylabel("V(m/s)")
plt.xlabel("Fan speed (Hz)")
plt.subplot(3,2,2)
plt.title("Ux Std")
plt.scatter(fans, ux_stds)
plt.ylim(0, 0.5)
plt.ylabel("V(m/s)")
plt.xlabel("Fan speed (Hz)")
plt.subplot(3,2,3)
plt.title("Uy Mean")
plt.scatter(fans, uy_means)
plt.ylim(-0.5, 0.5)
plt.ylabel("V(m/s)")
plt.xlabel("Fan speed (Hz)")
plt.subplot(3,2,4)
plt.title("Uy Std")
plt.scatter(fans, uy_stds)
plt.ylim(0, 0.5)
plt.ylabel("V(m/s)")
plt.xlabel("Fan speed (Hz)")
plt.subplot(3,2,5)
plt.title("Uz Mean")
plt.scatter(fans, uz_means)
plt.ylim(-0.5, 0.5)
plt.ylabel("V(m/s)")
plt.xlabel("Fan speed (Hz)")
plt.subplot(3,2,6)
plt.title("Uz Std")
plt.scatter(fans, uz_stds)
plt.ylim(0, 0.5)
plt.ylabel("V(m/s)")
plt.xlabel("Fan speed (Hz)")
plt.savefig(f"/home/craiglagegit/DATA/Anemom_vs_Fan_Speed_23Nov22.pdf")


In [ ]:
len(fans)